In [20]:
import os

DB_PATH = r'./data.db'
DATA_DIR_PATH = r"E:\paper\data_no_ads"
EMBEDDING_MODEL_PATH = r"C:\Users\wind\.cache\modelscope\hub\models\Qwen\Qwen3-Embedding-0___6B"
CROSSENCODER_MODEL_PATH = r"BAAI/bge-reranker-v2-m3"

In [21]:
def split_into_chunks(data_path):
    try:
        with open(data_path, 'r', encoding='utf-8') as f:
            data = f.readlines()
        data = data[2:-1]
        chunks = [''.join(data[i:i+5]) for i in range(0,len(data),5)]
        return chunks
    except FileNotFoundError:
        print(f"文件 {data_path} 不存在")
        return None
    
def get_dir_count(path = DATA_DIR_PATH):
    try:
        count = 0
        with os.scandir(path) as entries:
            for entry in entries:
                if entry.is_file() and entry.name.startswith('data_') and entry.name.endswith('.txt'):
                    count += 1
        return count
    except FileNotFoundError:
        print(f"文件夹 {path} 不存在")
        return None

In [22]:
def split_chapter_chunks(data_path):
    try:
        with open(data_path, 'r', encoding='utf-8') as f:
            data = f.readlines()
        chunks = data[:-1]
        chunks = ''.join(chunks)
        return chunks
    except FileNotFoundError:
        print(f"文件 {data_path} 不存在")
        return None
    
def get_dir_count(path = DATA_DIR_PATH):
    try:
        count = 0
        with os.scandir(path) as entries:
            for entry in entries:
                if entry.is_file() and entry.name.startswith('data_') and entry.name.endswith('.txt'):
                    count += 1
        return count
    except FileNotFoundError:
        print(f"文件夹 {path} 不存在")
        return None

In [23]:
test_data_path = '/root/yaolao/data/data_1.txt'
chunks = split_chapter_chunks(test_data_path)

文件 /root/yaolao/data/data_1.txt 不存在


In [24]:
file_count = get_dir_count()
chunks_all= []
for i in range(1, 8):
# for i in range(1, file_count//8):
    try:
        data_path = os.path.join(DATA_DIR_PATH, f'data_{i}.txt')
        chunks = split_chapter_chunks(data_path)
        chunks_all.append(chunks)
    except Exception as e:
        print(f"处理文件 {data_path} 时发生未知错误: {e}")
    
print(len(chunks_all))

7


In [25]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(EMBEDDING_MODEL_PATH)
def embed_chunks( chunks, batch_size=96):
    embeddings = model.encode( 
                              chunks, 
                              batch_size=batch_size
                              )
    return embeddings.tolist()

In [26]:
from tqdm import tqdm 
embeddings = [embed_chunks(chunks, 128) for chunks in tqdm(chunks_all, desc='生成嵌入', unit="chunks")]

生成嵌入: 100%|██████████| 7/7 [02:05<00:00, 17.88s/chunks]


In [27]:
import chromadb

chromadb_client = chromadb.PersistentClient(path=DB_PATH)
chromadb_collection = chromadb_client.get_or_create_collection(name="my_collection")

def save_embeddings(chunks: list[str],embeddings: list[list[float]], batch_size=1000):
    ids = [str(i) for i in range(len(chunks))]
    
    for i in tqdm(range(0, len(chunks), batch_size), desc="Saving embeddings"):
        batch_chunks = chunks[i:i + batch_size]
        batch_embeddings = embeddings[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]
        
        chromadb_collection.add(
            documents=batch_chunks,
            embeddings=batch_embeddings,
            ids=batch_ids
        )

def retrieve(query: str, top_k:int):
    query_embedding = model.encode(query)
    results = chromadb_collection.query(
        query_embeddings =[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]

In [28]:
save_embeddings(chunks_all,embeddings)

Saving embeddings: 100%|██████████| 1/1 [00:00<00:00, 22.73it/s]


In [ ]:
import re
query = "我的父亲叫什么名字?"
query = re.sub(r"我","萧炎", query)
query = re.sub(r"你|您", "药老", query)
retrieve_chunks = retrieve(query, 5)
for i, chunk in enumerate(retrieve_chunks):
    print(f"第{i+1}个相关段落：\n{chunk[:50]}\n")

萧炎的父亲叫什么名字?
第1个相关段落：
正文 第四章 云岚宗
 
大厅中,萧战以及三位长老,正在颇为热切的与那位陌生老者交谈着,不过这位老者

第2个相关段落：
正文 第二章 斗气大陆
 
月如银盘，漫天繁星。
山崖之颠，萧炎斜躺在草地之上，嘴中叼中一根青草，微

第3个相关段落：
正文 第三章 客人
 
床榻之上，少年闭目盘腿而坐，双手在身前摆出奇异的手印，胸膛轻微起伏，一呼一吸

第4个相关段落：
正文 第一章 陨落的天才
 
“斗之力，三段！”
望着测验魔石碑上面闪亮得甚至有些刺眼的五个大字，少

第5个相关段落：
正文 第七章 休！
 
与纳兰嫣然所期待的有些不同，在她话出之后，面前的少年，身体猛的剧烈颤抖了起来



In [30]:
from sentence_transformers import CrossEncoder

def rerank(query: str,retrieved_chunks: list[str] , top_k:int):
    cross_encoder = CrossEncoder(CROSSENCODER_MODEL_PATH)
    pairs = [(query, chunk) for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)
    
    chuck_with_score = [(chunk, score)
                        for chunk, score in zip(retrieved_chunks, scores)]
    chuck_with_score.sort(key=lambda x: x[1], reverse=True)

    return [chunk for chunk, _ in chuck_with_score][:top_k]

In [35]:
reranked_chunks = rerank(query, retrieve_chunks, 2)
for i,chunk in enumerate(reranked_chunks):
    print(f"第{i+1}个相关段落：\n{chunk[:50]}\n")

第1个相关段落：
正文 第二章 斗气大陆
 
月如银盘，漫天繁星。
山崖之颠，萧炎斜躺在草地之上，嘴中叼中一根青草，微

第2个相关段落：
正文 第七章 休！
 
与纳兰嫣然所期待的有些不同，在她话出之后，面前的少年，身体猛的剧烈颤抖了起来



In [36]:
print(reranked_chunks[0])

正文 第二章 斗气大陆
 
月如银盘，漫天繁星。
山崖之颠，萧炎斜躺在草地之上，嘴中叼中一根青草，微微嚼动，任由那淡淡的苦涩在嘴中弥漫开来…
举起有些白皙的手掌，挡在眼前，目光透过手指缝隙，遥望着天空上那轮巨大的银月。
“唉…”想起下午的测试，萧炎轻叹了一口气，懒懒的抽回手掌，双手枕着脑袋，眼神有些恍惚…
“十五年了呢…”低低的自喃声，忽然毫无边际的从少年嘴中轻吐了出来。
在萧炎的心中，有一个仅有他自己知道的秘密：他并不是这个世界的人，或者说，萧炎的灵魂，并不属于这个世界，他来自一个名叫地球的蔚蓝星球，至于为什么会来到这里，这种离奇经过，他也无法解释，不过在生活了一段时间之后，他还是后知后觉的明白了过来：他穿越了！
随着年龄的增长，对这块大陆，萧炎也是有了些模糊的了解…
大陆名为斗气大陆，大陆上并没有小说中常见的各系魔法，而斗气，才是大陆的唯一主调！
在这片大陆上，斗气的修炼，几乎已经在无数代人的努力之下，发展到了巅峰地步，而且由于斗气的不断繁衍，最后甚至扩散到了民间之中，这也导致，斗气，与人类的日常生活，变得息息相关，如此，斗气在大陆中的重要性，更是变得无可替代！
因为斗气的极端繁衍，同时也导致从这条主线中分化出了无数条斗气修炼之法，所谓手有长短，分化出来的斗气修炼之法，自然也是有强有弱。
经过归纳统计，斗气大陆将斗气功法的等级，由高到低分为四阶十二级：天.地.玄.黄！
而每一阶，又分初，中，高三级！
修炼的斗气功法等级的高低，也是决定日后成就高低的关键，比如修炼玄阶中级功法的人，自然要比修炼黄阶高级功法的同等级的人要强上几分。
斗气大陆，分辩强弱，取决于三种条件。
首先，最重要的，当然是自身的实力，如果本身实力只有一星斗者级别，那就算你修炼的是天阶高级的稀世功法，那也难以战胜一名修炼黄阶功法的斗师。
其次，便是功法！同等级的强者，如果你的功法等级较之对方要高级许多，那么在比试之时，种种优势，一触既知。
最后一种，名叫斗技！
顾名思义，这是一种发挥斗气的特殊技能，斗技在大陆之上，也有着等级之分，总的说来，同样也是分为天地玄黄四级。
斗气大陆斗技数不胜数，不过一般流传出来的大众斗技，大多都只是黄级左右，想要获得更高深的斗技，便必须加入宗派，或者大陆上的斗气学院。
当然，一些依靠奇遇所得到前人遗留而下的功法，或者有着自己相配套的斗技，这种由功法衍变而出的斗技，互相配合